In [1]:
import random
import glob
import os
import sys
import json
import math
import numpy as np
import pandas as pd
import sklearn
from sklearn.neighbors import NearestNeighbors
from pathlib import Path
import lightgbm as lgb
from typing import Iterable, Dict, Set, List
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras

In [2]:
pd.set_option("use_inf_as_na", True)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 1000)
INPUT = '/kaggle/input'
DATA = f'{INPUT}/shopee-product-matching'
OUTPUT = '/kaggle/temp'
RESOURCE_DIR = f'{INPUT}/shopee-product-matching-lib/kaggle-shopee-product-matching-1.0'
#LGB_MODEL_DIR = f'{RESOURCE_DIR}/models/lgb/20210220_213935'
#LGB_MODEL_DIR = f'{RESOURCE_DIR}/models/lgb/20210220_130330'
#MLP_MODEL_DIR = f'{RESOURCE_DIR}/models/mlp_20210222_221918'
#FEATURES_DIR = f'{RESOURCE_DIR}/features'
sys.path.append(f'{INPUT}/sgcharts-ml/src')
sys.path.append(f"{INPUT}/sentence-transformers/sentence-transformers-1.0.4")
sys.path.append(f'{RESOURCE_DIR}/src')

In [3]:
from sentence_transformers import SentenceTransformer
import mylib
import scml
scml.seed_everything()
test = pd.read_csv(f"{DATA}/test.csv", engine="c", low_memory=False)

# phash embedding

In [4]:
test["phash_matches"] = mylib.phash_matches(test, threshold=0.3)

# SBERT sentence embedding 

In [5]:
posting_ids = test["posting_id"].tolist()
sentences = test["title"].tolist()
test["sbert_matches"] = mylib.sbert_matches(
    model_path=f"{RESOURCE_DIR}/pretrained/sentence-transformers/stsb-distilbert-base",
    sentences=sentences,
    posting_ids=posting_ids,
    threshold=0.6
)

# Submission

In [6]:
cols = ["phash_matches", "sbert_matches"]
test["matches"] = test.apply(mylib.combine_as_string(cols), axis=1)
sub = test[["posting_id", "matches"]]
sub.head()

,posting_id,matches
0,test_2255846744,test_2255846744
1,test_3588702337,test_3588702337
2,test_4015706929,test_4015706929


In [7]:
sub.to_csv("submission.csv", index = False)

# Debug

In [8]:
!pip list

Package                        Version             Location
------------------------------ ------------------- --------------
absl-py                        0.12.0
adal                           1.2.6
affine                         2.3.0
aiobotocore                    1.2.2
aiohttp                        3.7.3
aiohttp-cors                   0.7.0
aioitertools                   0.7.1
aioredis                       1.3.1
albumentations                 0.5.2
alembic                        1.5.8
allennlp                       2.2.0
altair                         4.1.0
annoy                          1.17.0
ansiwrap                       0.8.4
appdirs                        1.4.4
argon2-cffi                    20.1.0
arrow                          0.17.0
arviz                          0.11.2
asn1crypto                     1.4.0
astropy                        4.2
astunparse                     1.6.3
async-generator                1.10
async-timeout                  3.0.1
attrs                